In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pymannkendall

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# this is implemented from STATE ACTION ANALYSIS FILE : https://colab.research.google.com/drive/1n9nJdd84etLey3piTKkRJmttpZlW1gor#scrollTo=bgGiHn-LobHm

#GET ALL STATE ACTION MODEL FOR NDSI 2D AND NDSI 3D
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/stateActionModel/"
path=(str(global_drive_path) +"/ndsi-3d")
fileNames_NDSI_3D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(path, filename))
        fileNames_NDSI_3D.append(filename)
    else:
        continue
path=(str(global_drive_path) +"/ndsi-2d")
fileNames_NDSI_2D=[]
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filePath=(os.path.join(path, filename))
        fileNames_NDSI_2D.append(filename)
    else:
        continue
matchingName=[]
df_twoD=[]
df_threeD=[]
path_twoD=(str(global_drive_path) +"/ndsi-2d")
path_threeD=(str(global_drive_path) +"/ndsi-3d")
for i in range(len(fileNames_NDSI_2D)):
  for j in range(len(fileNames_NDSI_3D)):
  # if fileNames_NDSI_2D[i].removesuffix('taskname_ndsi-2d-task_') == fileNames_NDSI_3D[i].removesuffix('taskname_ndsi-3d-task_'):
   if fileNames_NDSI_2D[i][22:] == fileNames_NDSI_3D[j][22:]:
      #print(os.path.join(path, str(fileNames_NDSI_2D[i])))
      df=pd.read_csv(os.path.join(path_twoD, str(fileNames_NDSI_2D[i])),sep=',')
      df_twoD.append(df)
      df=pd.read_csv(os.path.join(path_threeD, str(fileNames_NDSI_3D[j])),sep=',')
      df_threeD.append(df)
      matchingName.append(fileNames_NDSI_2D[i])
#STORE ALL USERS NAME
users=[]
for name in matchingName:
    users.append(name[22:-4:1])

In [4]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')
      
      self.pValue = adfTest[1]
      
      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False
      
      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)
    
    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries)
      
      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]
      
      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [5]:
def make_numerical_dataframe(df):
  df['State_Action']=(df['Action']+ df['State'])
 
  df['Action']=pd.factorize(df['Action'])[0]
  df['State']=pd.factorize(df['State'])[0]
  state_action_correlated_series=pd.factorize(df['State_Action'])[0]
 
  action_only_series=df['Action'].values
  state_only_series=df['State'].values
  options=[state_action_correlated_series,action_only_series,state_only_series]
  return options

In [9]:
#3D Mann Kendall
saveDir=(str(global_drive_path) +"StationarityTests/ndsi-3d/ActionBased")
combined_final_dataframe_3d = pd.DataFrame(columns=['User','ActionNumber','Subtask_IdleTime','MKT Stationary_State_Action'])
array_of_dataframes=[]
for j in tqdm(range(len(df_threeD))):
  
  all_states_dataframes=[]
  df_Navigation=df_threeD[j].loc[df_threeD[j]['State'] == 'Navigation']
  all_states_dataframes.append(df_Navigation)
  df_Sensemaking=df_threeD[j].loc[df_threeD[j]['State'] == 'Sensemaking']
  all_states_dataframes.append(df_Sensemaking)
  df_Answering=df_threeD[j].loc[df_threeD[j]['State'] == 'Answering']
  all_states_dataframes.append(df_Answering)
  df_Foraging=df_threeD[j].loc[df_threeD[j]['State'] == 'Foraging']
  all_states_dataframes.append(df_Foraging)
  print(j)
  print(users[j])
  #running MKT on one user after dividing data based on states
  if (len(all_states_dataframes) == 4):
    actiondf=Subset_screenshot_time(users[j],all_states_dataframes)
    array_of_dataframes.append(actiondf)
    
  else:
    print("failed for user", user[j])
array_of_dataframes = pd.concat(array_of_dataframes)
userfilename= "All Users_MaxIdleTime_MKT_sliding_.xlsx"
array_of_dataframes.to_excel(os.path.join(saveDir,userfilename), index=False)
   

  0%|          | 0/19 [00:00<?, ?it/s]

0
userid_82316e37-1117-4663-84b4-ddb6455c83b2



  5%|▌         | 1/19 [00:00<00:03,  5.49it/s]

For action: ############ 0
[1]
For action: ############ 1
[1 2]
For action: ############ 2
[1]
For action: ############ 3
[1]
1
userid_d6555293-35c7-4c27-b522-38d67d2d1b1a



  0%|          | 0/4 [00:00<?, ?it/s]

...

[2]
For action: ############ 2
[2]
For action: ############ 3
[1]


18
userid_93be0a02-3f7a-46c9-b526-aa956ed00856



  0%|          | 0/4 [00:00<?, ?it/s]

For action: ############ 0
[2]
For action: ############ 1
[2]
For action: ############ 2
[2]
For action: ############ 3
[1]


100%|██████████| 19/19 [00:02<00:00,  7.16it/s]


In [6]:

#3D Mann Kendall
def Subset_screenshot_time(user,all_states_dataframe):
  failed_files=[]
  final_dataframe_3d = pd.DataFrame(columns=['User','ActionNumber','Subtask_IdleTime','MKT Stationary_State_Action'])
  for i in tqdm(range(len(all_states_dataframe))):
    subtask=0
    length_parsedFile=0
    print("For action: ############",i)

    window_dataframe=all_states_dataframe[i]

    subtask_values=window_dataframe["Subtask_Idle_Time"].unique()
    print(subtask_values)
    original_fileLength=len(window_dataframe)

    while ((length_parsedFile < original_fileLength) and (subtask <= len(subtask_values))):
      
      window_dataframe_subset=window_dataframe.loc[window_dataframe['Subtask_Idle_Time'] == subtask_values[subtask]]
      length_parsedFile+=len(window_dataframe_subset)
      #print("Length of file",len(window_dataframe_subset), " in ","Subtask: ",subtask_values[subtask])
      #creating series
      state_action_correlated_series=window_dataframe_subset['StateActionProbab'].values

      three_series=[state_action_correlated_series]
      three_series_results=[user,i,subtask_values[subtask]]
      for series in (three_series):
        if len(series)>2:
          sTest_state = StationarityTests()
          sTest_state.Mann_Kendall_Stationarity_Test(series, printResults = False)
          three_series_results.append(sTest_state.trend)
          
        else:
          three_series_results.append("Not Enough Data")
          failed_files.append(user)
      #adding results per subtask per user to dataframe
      df_length = len(final_dataframe_3d)
      final_dataframe_3d.loc[df_length] = three_series_results
      subtask +=1
  return final_dataframe_3d
    

In [ ]:
window_dataframe

NameError: ignored

In [ ]:
#2D Mann Kendall
saveDir=(str(global_drive_path) +"StationarityTests/ndsi-2d/MKT_Sliding")
failed_files=[]
for i in tqdm(range(len(df_twoD))):
  final_dataframe_3d = pd.DataFrame(columns=['User','MKT Stationary_State_Action'])
  subtask=0
  length_parsedFile=0


  window_dataframe=(df_twoD[i])
  window_dataframe['State_Action']=(window_dataframe['Action']+ window_dataframe['State'])
  window_dataframe['Action']=pd.factorize(window_dataframe['Action'])[0]
  window_dataframe['State']=pd.factorize(window_dataframe['State'])[0]
  window_dataframe['State_Action']=pd.factorize(window_dataframe['State_Action'])[0]
  subtask_values=window_dataframe["Subtask_Idle_Time"].unique()
  original_fileLength=len(window_dataframe)

  print("Starting for user", users[i])
  while ((length_parsedFile < original_fileLength) and (subtask <= len(subtask_values))):
    
    window_dataframe_subset=window_dataframe.loc[window_dataframe['Subtask_Idle_Time'] == subtask_values[subtask]]
    length_parsedFile+=len(window_dataframe_subset)
    print("Length of file",len(window_dataframe_subset), " in ","Subtask: ",subtask_values[subtask])
    #creating series
    state_action_correlated_series=window_dataframe_subset['State_Action'].values
    # action_only_series=window_dataframe_subset['Action'].values
    # state_only_series=window_dataframe_subset['State'].values
    three_series=[state_action_correlated_series]
    three_series_results=[subtask_values[subtask]]
    for series in (three_series):
      if len(series)>2:
        sTest_state = StationarityTests()
        sTest_state.Mann_Kendall_Stationarity_Test(series, printResults = False)
        three_series_results.append(sTest_state.trend)
        
      else:
        three_series_results.append("Not Enough Data")
        print("Failed for user", users[i])
        failed_files.append(users[i])
    df_length = len(final_dataframe_3d)
    final_dataframe_3d.loc[df_length] = three_series_results
    userfilename= str(users[i]) + "MKT_sliding.csv"
    final_dataframe_3d.to_csv(os.path.join(saveDir,userfilename), index=False)
    subtask +=1
    

  0%|          | 0/19 [00:00<?, ?it/s]

Starting for user userid_ac5b65d7-f260-4534-9fae-d998b726ed32
Length of file 42  in  Subtask:  1


  5%|▌         | 1/19 [00:00<00:15,  1.14it/s]

Length of file 51  in  Subtask:  2
Length of file 19  in  Subtask:  3
Starting for user userid_82316e37-1117-4663-84b4-ddb6455c83b2
Length of file 11  in  Subtask:  1


 11%|█         | 2/19 [00:01<00:10,  1.58it/s]

...

Length of file 13  in  Subtask:  2
Length of file 10  in  Subtask:  3
Starting for user userid_44968286-f204-4ad6-a9b5-d95b38e97866
Length of file 23  in  Subtask:  1
Length of file 56  in  Subtask:  2
Length of file 3  in  Subtask:  3
Length of file 5  in  Subtask:  4
Starting for user userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d
Length of file 22  in  Subtask:  1


 95%|█████████▍| 18/19 [00:07<00:00,  2.88it/s]

Length of file 17  in  Subtask:  2
Length of file 10  in  Subtask:  3
Length of file 10  in  Subtask:  4
Starting for user userid_8b544d24-3274-4bb0-9719-fd2bccc87b02
Length of file 22  in  Subtask:  1


100%|██████████| 19/19 [00:07<00:00,  2.44it/s]

Length of file 35  in  Subtask:  2
Length of file 2  in  Subtask:  3
Failed for user userid_8b544d24-3274-4bb0-9719-fd2bccc87b02
Length of file 15  in  Subtask:  4


In [ ]:
failed_files

['userid_bc9f9425-e867-4876-a5d9-791229916767',
 'userid_3abeecbe-327a-441e-be2a-0dd3763c1d45',
 'userid_e4221e48-0930-40db-af3c-f4de1ed87582',
 'userid_8b544d24-3274-4bb0-9719-fd2bccc87b02']